In [224]:
#importy
import numpy as np
import deepxde as dde


In [225]:
#stale
L = 2

In [226]:
#dziedzina
# geometry = dde.geometry.Hypercube(xmin=[-L/2, -L/2, 1, 1], xmax=[L/2, L/2, 2, 2])
g1 = dde.geometry.Hypercube(xmin=[-L/2, -L/2, 1, 1], xmax=[L/2, L/2, 2, 2])
g2 = dde.geometry.Hypercube(xmin=[-L/2, -L/2, 1, 1], xmax=[L/2, L/2, 1.01, 1.01])
geometry1 = dde.geometry.CSGIntersection(g1, g2)

g2 = dde.geometry.Hypercube(xmin=[-L/2, -L/2, 1.99, 1.99], xmax=[L/2, L/2, 2, 2])
geometry2 = dde.geometry.CSGIntersection(g1, g2)

geometry = dde.geometry.CSGUnion(geometry1, geometry2)

print(geometry.random_points(10))



[[-0.83376455  0.8188106   1.0077195   1.0095097 ]
 [ 0.50342727 -0.10283902  1.007241    1.0048175 ]
 [-0.97564256  0.40057164  1.0054275   1.0036246 ]
 [-0.6913259   0.15233746  1.9983838   1.9925638 ]
 [ 0.26335728  0.99711025  1.0022451   1.0014228 ]
 [ 0.3132765   0.328308    1.0017008   1.0000653 ]
 [-0.13261157 -0.1075706   1.9969563   1.9965959 ]
 [ 0.6623776   0.4817965   1.997635    1.9904597 ]
 [ 0.6202322  -0.23702967  1.9995329   1.9976605 ]
 [-0.6570463  -0.795247    1.9961498   1.996243  ]]


In [227]:
#funkcja
def pde(x, y):
    du_xx = dde.grad.hessian(y, x, i=0, j=0)
    du_yy = dde.grad.hessian(y, x, i=1, j=1)
    n1 = x[:, 2:3]
    n2 = x[:, 3:4]
    return 0.5 * (du_xx + du_yy) + (n1*n1 + n2*n2) * np.pi * y / (2 * L * L)



In [228]:
#warunki brzegowe
def boundaryFunction(x, on_boundary):
    if not on_boundary:
        return False
    if dde.utils.isclose(x[0], -L/2) or dde.utils.isclose(x[0], L/2):
        return True
    if dde.utils.isclose(x[1], -L/2) or dde.utils.isclose(x[1], L/2):
        return True
    return False


boundary_conditions = dde.icbc.DirichletBC(geometry, 
                                            lambda x: 0, 
                                            lambda x, on_boundary: boundaryFunction(x, on_boundary))


In [229]:
#dane
data = dde.data.PDE(geometry, 
                    pde, 
                    boundary_conditions, 
                    num_domain=4000,
                    num_boundary=2000,
                    num_test=1000
                    )

In [230]:
#siec
net = dde.nn.FNN([4]+[30]*3+[1],
                "tanh",
                "Glorot normal"
                )

In [231]:
#model
model = dde.Model(data, net)

model.compile("adam", lr=0.001)

Compiling model...
Building feed-forward neural network...
'build' took 0.094138 s



/home/filipjedrzejewski/.local/lib/python3.11/site-packages/deepxde/nn/tensorflow_compat_v1/fnn.py:116: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  return tf.layers.dense(


'compile' took 0.937915 s



In [232]:
#trenowanie
loss_history, train_state = model.train(iterations = 1000)

Training model...

0         [9.73e-02, 5.36e-02]    [9.29e-02, 5.36e-02]    []  


KeyboardInterrupt: 